In [29]:
import pandas as pd
import glob
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df_pl = pd.read_csv('/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/t20_pl_g.csv')

/Users/roumyadas/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (20,21) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [30]:
df_pl['phase'] = np.where(df_pl['ball']<7, 'pp', 
                        np.where(df_pl['ball']>=15, 'death',
                            'middle'))

df_pl['phase'].value_counts()

middle    174291
pp        150243
death      92035
Name: phase, dtype: int64

In [31]:
df = df_pl.sample(frac=1)

In [32]:
df_1 = df[df.innings==1]
df_1.shape

(217743, 61)

In [33]:
df_2 = df[df.innings==2]
df_2.shape

(198826, 61)

In [34]:
player_list = pd.read_excel('/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/player_list.xlsx',
                           sheet_name='season_01')

player_list.iloc[130:135]

players_list = player_list['Func_Name'].dropna().tolist()

#players_list = players_list.remove('nan')

print(type(players_list))

len(players_list)

<class 'list'>


221

In [35]:
#### innings 1 stuff, no run rate required

In [36]:
df_1_bo = df_1[df_1.bowler.isin(players_list)]

df_1_bo.shape

(111719, 61)

In [37]:
df_1_bo['runs_conceeded'] = df_1_bo['runs_off_bat']+df_1_bo['wides']+df_1_bo['noballs']

df_1_bo['isDotforbowler'] = np.where((df_1_bo['runs_conceeded']==0)&(df_1_bo['islegal']==1), 1, 0)

/var/folders/68/nlclwstn1zd28pll4j60q7zw0000gn/T/ipykernel_78239/2821836667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1_bo['runs_conceeded'] = df_1_bo['runs_off_bat']+df_1_bo['wides']+df_1_bo['noballs']
/var/folders/68/nlclwstn1zd28pll4j60q7zw0000gn/T/ipykernel_78239/2821836667.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1_bo['isDotforbowler'] = np.where((df_1_bo['runs_conceeded']==0)&(df_1_bo['islegal']==1), 1, 0)


In [38]:
df_1_bo.columns

Index(['match_id', 'season', 'start_date', 'venue', 'innings', 'ball',
       'batting_team', 'bowling_team', 'striker', 'non_striker', 'bowler',
       'runs_off_bat', 'extras', 'wides', 'noballs', 'byes', 'legbyes',
       'penalty', 'wicket_type', 'player_dismissed', 'other_wicket_type',
       'other_player_dismissed', 'year', 'legality', 'bowler_wicket',
       'islegal', 'isDotforBatter', 'isOne', 'isTwo', 'isThree',
       'isStrikeRotation', 'isFour', 'isSix', 'isBowlerWicket',
       'is_striker_Out', 'is_nonstriker_Out', 'isWicket', 'total_runs',
       'runs_scored', 'wickets_down', 'legal_balls_bowled', 'lbb_fl',
       'run_rate', 'target', 'legal_balls_remaining', 'runs_remaining',
       'reqd_run_rate', 'last_fow_run_rate', 'last_fow', 'partnership_runs',
       'striker_runs', 'non_striker_runs', 'isno', 'is_faced_by_batter',
       'striker_balls', 'non_striker_balls', 'fours', 'sixes',
       'striker_batting_position', 'non_striker_batting_position', 'phase',
      

In [39]:
bowler_stats = df_1_bo.groupby(['bowler','phase','innings']).agg(
    num_innings = ('match_id','nunique'),
    runs = ('runs_conceeded','sum'),
    balls = ('islegal' ,'sum'),
    wkts = ('isBowlerWicket','sum'),
    fours = ('isFour', 'sum'),
    sixes = ('isSix','sum'),
    dots = ('isDotforbowler','sum'),
    
    ones = ('isOne','sum'),
    twos = ('isTwo','sum'),
    threes = ('isThree','sum'),
    wides = ('wides','sum'),
    noballs = ('noballs','sum')
        
    
).reset_index()

bowler_stats['economy'] = 6*bowler_stats['runs']/bowler_stats['balls']
bowler_stats['strike_rate'] = bowler_stats['balls']/bowler_stats['wkts']
bowler_stats['bpb'] = bowler_stats['balls']/(bowler_stats['fours']+bowler_stats['sixes'])
bowler_stats['dot_%'] = 100*bowler_stats['dots']/bowler_stats['balls']

bowler_stats['wkt_pb'] = 1/bowler_stats['strike_rate']

bowler_stats['one_pb'] = bowler_stats['ones']/bowler_stats['balls']
bowler_stats['two_pb'] = bowler_stats['twos']/bowler_stats['balls']
bowler_stats['three_pb'] = bowler_stats['threes']/bowler_stats['balls']
bowler_stats['four_pb'] = bowler_stats['fours']/bowler_stats['balls']
bowler_stats['six_pb'] = bowler_stats['sixes']/bowler_stats['balls']
bowler_stats['dot_pb'] = bowler_stats['dots']/bowler_stats['balls']

bowler_stats['wide_pb'] = bowler_stats['wides']/bowler_stats['balls']
bowler_stats['no_pb'] = bowler_stats['noballs']/bowler_stats['balls']


pb_cols = [column for column in bowler_stats.columns if '_pb' in str(column)]
pb_cols_div = [column for column in bowler_stats.columns if '_pb' in str(column) and column!='wkt_pb']
print(len(pb_cols))


9


In [40]:
bowler_stats['pb_sum'] = bowler_stats[pb_cols].apply(lambda x: x.sum(), axis=1)

bowler_stats['wkt_prob'] = bowler_stats['wkt_pb']/1

for col in pb_cols_div:
    rem_sum = (bowler_stats['pb_sum']-bowler_stats['wkt_pb'])
    multiplier = (1-bowler_stats['wkt_prob'])/rem_sum
    bowler_stats[str(col).replace('_pb','_prob')] = bowler_stats[col]*multiplier

In [41]:
bowler_stats.tail(50)

,bowler,phase,innings,num_innings,runs,balls,wkts,fours,sixes,dots,ones,twos,threes,wides,noballs,economy,strike_rate,bpb,dot_%,wkt_pb,one_pb,two_pb,three_pb,four_pb,six_pb,dot_pb,wide_pb,no_pb,pb_sum,wkt_prob,one_prob,two_prob,three_prob,four_prob,six_prob,dot_prob,wide_prob,no_prob
383,TA Boult,death,1,73,1031.0,624,54,83,53,193,240,54,1,30.0,0.0,9.913462,11.555556,4.588235,30.929487,0.086538,0.384615,0.086538,0.001603,0.133013,0.084936,0.309295,0.048077,0.000000,1.134615,0.086538,0.335215,0.075423,0.001397,0.115929,0.074027,0.269569,0.041902,0.000000
384,TA Boult,middle,1,32,297.0,216,12,27,9,73,93,13,1,13.0,0.0,8.250000,18.000000,6.000000,33.796296,0.055556,0.430556,0.060185,0.004630,0.125000,0.041667,0.337963,0.060185,0.000000,1.115741,0.055556,0.383552,0.053615,0.004124,0.111354,0.037118,0.301067,0.053615,0.000000
385,TA Boult,pp,1,84,1290.0,1128,54,175,28,616,237,66,6,35.0,0.0,6.861702,20.888889,5.556650,54.609929,0.047872,0.210106,0.058511,0.005319,0.155142,0.024823,0.546099,0.031028,0.000000,1.078901,0.047872,0.194028,0.054033,0.004912,0.143269,0.022923,0.504308,0.028654,0.000000
386,TH David,death,1,4,43.0,24,3,3,1,7,8,4,1,6.0,0.0,10.750000,8.000000,6.000000,29.166667,0.125000,0.333333,0.166667,0.041667,0.125000,0.041667,0.291667,0.250000,0.000000,1.375000,0.125000,0.233333,0.116667,0.029167,0.087500,0.029167,0.204167,0.175000,0.000000
387,TH David,middle,1,20,275.0,174,5,22,9,42,84,17,0,15.0,0.0,9.482759,34.800000,5.612903,24.137931,0.028736,0.482759,0.097701,0.000000,0.126437,0.051724,0.241379,0.086207,0.000000,1.114943,0.028736,0.431673,0.087362,0.000000,0.113057,0.046251,0.215837,0.077084,0.000000
388,TH David,pp,1,18,160.0,122,3,12,4,36,62,7,1,9.0,0.0,7.868852,40.666667,7.625000,29.508197,0.024590,0.508197,0.057377,0.008197,0.098361,0.032787,0.295082,0.073770,0.000000,1.098361,0.024590,0.461644,0.052121,0.007446,0.089351,0.029784,0.268052,0.067013,0.000000
389,TM Head,middle,1,5,54.0,36,3,4,2,9,16,5,0,0.0,0.0,9.000000,12.000000,6.000000,25.000000,0.083333,0.444444,0.138889,0.000000,0.111111,0.055556,0.250000,0.000000,0.000000,1.083333,0.083333,0.407407,0.127315,0.000000,0.101852,0.050926,0.229167,0.000000,0.000000
390,TM Head,pp,1,4,47.0,30,0,5,1,8,12,4,0,1.0,0.0,9.400000,inf,5.000000,26.666667,0.000000,0.400000,0.133333,0.000000,0.166667,0.033333,0.266667,0.033333,0.000000,1.033333,0.000000,0.387097,0.129032,0.000000,0.161290,0.032258,0.258065,0.032258,0.000000
391,TU Deshpande,death,1,33,449.0,273,28,30,25,90,104,26,0,20.0,3.0,9.868132,9.750000,4.963636,32.967033,0.102564,0.380952,0.095238,0.000000,0.109890,0.091575,0.329670,0.073260,0.010989,1.194139,0.102564,0.313199,0.078300,0.000000,0.090346,0.075288,0.271038,0.060231,0.009035
392,TU Deshpande,middle,1,21,209.0,156,14,22,6,56,63,8,1,3.0,0.0,8.038462,11.142857,5.571429,35.897436,0.089744,0.403846,0.051282,0.006410,0.141026,0.038462,0.358974,0.019231,0.000000,1.108974,0.089744,0.360668,0.045799,0.005725,0.125947,0.034349,0.320593,0.017175,0.000000


In [42]:
prob_cols = [column for column in bowler_stats.columns if '_prob' in str(column)]

for col in bowler_stats.columns[[0,1,2,3,4,5,6,15,16]]:
    prob_cols.append(col)

bowler_stats_trimmed = bowler_stats[prob_cols]

In [43]:
bowler_stats_trimmed[bowler_stats_trimmed.bowler=='VG Arora']

,wkt_prob,one_prob,two_prob,three_prob,four_prob,six_prob,dot_prob,wide_prob,no_prob,bowler,phase,innings,num_innings,runs,balls,wkts,economy,strike_rate
405,0.020833,0.231439,0.160227,0.0,0.195833,0.035606,0.231439,0.106818,0.017803,VG Arora,death,1,7,94.0,48,1,11.750000,48.0
406,0.000000,0.166667,0.000000,0.0,0.500000,0.000000,0.333333,0.000000,0.000000,VG Arora,middle,1,1,13.0,6,0,13.000000,inf
407,0.059524,0.259266,0.045753,0.0,0.106757,0.020335,0.427027,0.061004,0.020335,VG Arora,pp,1,11,193.0,168,10,6.892857,16.8


In [44]:
bowler_stats_trimmed.to_csv('/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/bowler_stats_inn1.csv',
                           index=None)

In [45]:
#### innings 2 stuff, bucketing based on run rate required, balls remain, wickets lost

In [46]:
df_2_bo = df_2[df_2.bowler.isin(players_list)]

df_2_bo.shape

(107528, 61)

In [47]:
df_2_bo['runs_conceeded'] = df_2_bo['runs_off_bat']+df_2_bo['wides']+df_2_bo['noballs']

df_2_bo['isDotforbowler'] = np.where((df_2_bo['runs_conceeded']==0)&(df_2_bo['islegal']==1), 1, 0)

/var/folders/68/nlclwstn1zd28pll4j60q7zw0000gn/T/ipykernel_78239/3912845939.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2_bo['runs_conceeded'] = df_2_bo['runs_off_bat']+df_2_bo['wides']+df_2_bo['noballs']
/var/folders/68/nlclwstn1zd28pll4j60q7zw0000gn/T/ipykernel_78239/3912845939.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2_bo['isDotforbowler'] = np.where((df_2_bo['runs_conceeded']==0)&(df_2_bo['islegal']==1), 1, 0)


In [48]:
df_2_bo['nrr_phase'] = np.where(df_2_bo['reqd_run_rate']>=10, 'easy', 
                        np.where(df_2_bo['reqd_run_rate']<=8, 'crucial',
                            'moderate'))

df_2_bo['wkt_phase'] = np.where(df_2_bo['wickets_down']>=7, 'easy', 
                        np.where(df_2_bo['wickets_down']<=3, 'crucial',
                            'medium'))

df_2_bo['nrr_phase'].value_counts(), df_2_bo['wkt_phase'].value_counts()

/var/folders/68/nlclwstn1zd28pll4j60q7zw0000gn/T/ipykernel_78239/2809952967.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2_bo['nrr_phase'] = np.where(df_2_bo['reqd_run_rate']>=10, 'easy',
/var/folders/68/nlclwstn1zd28pll4j60q7zw0000gn/T/ipykernel_78239/2809952967.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2_bo['wkt_phase'] = np.where(df_2_bo['wickets_down']>=7, 'easy',


(easy        45441
 crucial     34201
 moderate    27886
 Name: nrr_phase, dtype: int64,
 crucial    72939
 medium     26269
 easy        8320
 Name: wkt_phase, dtype: int64)

In [49]:
df_2_bo.columns
#wickets_down, reqd_run_rate

Index(['match_id', 'season', 'start_date', 'venue', 'innings', 'ball',
       'batting_team', 'bowling_team', 'striker', 'non_striker', 'bowler',
       'runs_off_bat', 'extras', 'wides', 'noballs', 'byes', 'legbyes',
       'penalty', 'wicket_type', 'player_dismissed', 'other_wicket_type',
       'other_player_dismissed', 'year', 'legality', 'bowler_wicket',
       'islegal', 'isDotforBatter', 'isOne', 'isTwo', 'isThree',
       'isStrikeRotation', 'isFour', 'isSix', 'isBowlerWicket',
       'is_striker_Out', 'is_nonstriker_Out', 'isWicket', 'total_runs',
       'runs_scored', 'wickets_down', 'legal_balls_bowled', 'lbb_fl',
       'run_rate', 'target', 'legal_balls_remaining', 'runs_remaining',
       'reqd_run_rate', 'last_fow_run_rate', 'last_fow', 'partnership_runs',
       'striker_runs', 'non_striker_runs', 'isno', 'is_faced_by_batter',
       'striker_balls', 'non_striker_balls', 'fours', 'sixes',
       'striker_batting_position', 'non_striker_batting_position', 'phase',
      

In [50]:
bowler_stats = df_2_bo.groupby(['bowler','innings','phase','nrr_phase','wkt_phase']).agg(
    num_innings = ('match_id','nunique'),
    runs = ('runs_conceeded','sum'),
    balls = ('islegal' ,'sum'),
    wkts = ('isBowlerWicket','sum'),
    fours = ('isFour', 'sum'),
    sixes = ('isSix','sum'),
    dots = ('isDotforbowler','sum'),
    
    ones = ('isOne','sum'),
    twos = ('isTwo','sum'),
    threes = ('isThree','sum'),
    wides = ('wides','sum'),
    noballs = ('noballs','sum')
        
    
).reset_index()

bowler_stats['economy'] = 6*bowler_stats['runs']/bowler_stats['balls']
bowler_stats['strike_rate'] = bowler_stats['balls']/bowler_stats['wkts']
bowler_stats['bpb'] = bowler_stats['balls']/(bowler_stats['fours']+bowler_stats['sixes'])
bowler_stats['dot_%'] = 100*bowler_stats['dots']/bowler_stats['balls']

bowler_stats['wkt_pb'] = 1/bowler_stats['strike_rate']

bowler_stats['one_pb'] = bowler_stats['ones']/bowler_stats['balls']
bowler_stats['two_pb'] = bowler_stats['twos']/bowler_stats['balls']
bowler_stats['three_pb'] = bowler_stats['threes']/bowler_stats['balls']
bowler_stats['four_pb'] = bowler_stats['fours']/bowler_stats['balls']
bowler_stats['six_pb'] = bowler_stats['sixes']/bowler_stats['balls']
bowler_stats['dot_pb'] = bowler_stats['dots']/bowler_stats['balls']

bowler_stats['wide_pb'] = bowler_stats['wides']/bowler_stats['balls']
bowler_stats['no_pb'] = bowler_stats['noballs']/bowler_stats['balls']


pb_cols = [column for column in bowler_stats.columns if '_pb' in str(column)]
pb_cols_div = [column for column in bowler_stats.columns if '_pb' in str(column) and column!='wkt_pb']
print(len(pb_cols))


9


In [51]:
bowler_stats['pb_sum'] = bowler_stats[pb_cols].apply(lambda x: x.sum(), axis=1)

bowler_stats['wkt_prob'] = bowler_stats['wkt_pb']/1

for col in pb_cols_div:
    rem_sum = (bowler_stats['pb_sum']-bowler_stats['wkt_pb'])
    multiplier = (1-bowler_stats['wkt_prob'])/rem_sum
    bowler_stats[str(col).replace('_pb','_prob')] = bowler_stats[col]*multiplier

In [52]:
bowler_stats.tail(50)

,bowler,innings,phase,nrr_phase,wkt_phase,num_innings,runs,balls,wkts,fours,sixes,dots,ones,twos,threes,wides,noballs,economy,strike_rate,bpb,dot_%,wkt_pb,one_pb,two_pb,three_pb,four_pb,six_pb,dot_pb,wide_pb,no_pb,pb_sum,wkt_prob,one_prob,two_prob,three_prob,four_prob,six_prob,dot_prob,wide_prob,no_prob
2162,YS Chahal,2,middle,easy,medium,37,346.0,301,32,13,19,123,126,19,1,12.0,1.0,6.897010,9.406250,9.406250,40.863787,0.106312,0.418605,0.063123,0.003322,0.043189,0.063123,0.408638,0.039867,0.003322,1.149502,0.106312,0.358614,0.054077,0.002846,0.037000,0.054077,0.350075,0.034154,0.002846
2163,YS Chahal,2,middle,moderate,crucial,33,450.0,307,12,23,25,89,145,24,1,12.0,0.0,8.794788,25.583333,6.395833,28.990228,0.039088,0.472313,0.078176,0.003257,0.074919,0.081433,0.289902,0.039088,0.000000,1.078176,0.039088,0.436778,0.072294,0.003012,0.069282,0.075307,0.268091,0.036147,0.000000
2164,YS Chahal,2,middle,moderate,medium,8,47.0,43,3,4,1,18,16,3,1,0.0,0.0,6.558140,14.333333,8.600000,41.860465,0.069767,0.372093,0.069767,0.023256,0.093023,0.023256,0.418605,0.000000,0.000000,1.069767,0.069767,0.346133,0.064900,0.021633,0.086533,0.021633,0.389400,0.000000,0.000000
2165,YS Chahal,2,pp,crucial,crucial,23,168.0,154,5,13,6,70,52,13,0,2.0,0.0,6.545455,30.800000,8.105263,45.454545,0.032468,0.337662,0.084416,0.000000,0.084416,0.038961,0.454545,0.012987,0.000000,1.045455,0.032468,0.322511,0.080628,0.000000,0.080628,0.037213,0.434149,0.012404,0.000000
2166,YS Chahal,2,pp,easy,crucial,27,200.0,175,10,18,6,66,81,4,0,3.0,0.0,6.857143,17.500000,7.291667,37.714286,0.057143,0.462857,0.022857,0.000000,0.102857,0.034286,0.377143,0.017143,0.000000,1.074286,0.057143,0.429053,0.021188,0.000000,0.095345,0.031782,0.349599,0.015891,0.000000
2167,YS Chahal,2,pp,easy,medium,1,0.0,1,1,0,0,1,0,0,0,0.0,0.0,0.000000,1.000000,inf,100.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2168,YS Chahal,2,pp,moderate,crucial,32,218.0,177,4,15,11,70,78,3,0,8.0,0.0,7.389831,44.250000,6.807692,39.548023,0.022599,0.440678,0.016949,0.000000,0.084746,0.062147,0.395480,0.045198,0.000000,1.067797,0.022599,0.412093,0.015850,0.000000,0.079249,0.058116,0.369827,0.042266,0.000000
2169,YS Chahal,2,pp,moderate,medium,1,1.0,3,1,0,0,2,1,0,0,0.0,0.0,2.000000,3.000000,inf,66.666667,0.333333,0.333333,0.000000,0.000000,0.000000,0.000000,0.666667,0.000000,0.000000,1.333333,0.333333,0.222222,0.000000,0.000000,0.000000,0.000000,0.444444,0.000000,0.000000
2170,Yash Dayal,2,death,crucial,crucial,3,27.0,18,2,3,0,4,9,2,0,2.0,0.0,9.000000,9.000000,6.000000,22.222222,0.111111,0.500000,0.111111,0.000000,0.166667,0.000000,0.222222,0.111111,0.000000,1.222222,0.111111,0.400000,0.088889,0.000000,0.133333,0.000000,0.177778,0.088889,0.000000
2171,Yash Dayal,2,death,crucial,easy,1,6.0,1,0,0,1,0,0,0,0,0.0,0.0,36.000000,inf,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000


In [53]:
prob_cols = [column for column in bowler_stats.columns if '_prob' in str(column)]

for col in bowler_stats.columns[[0,1,2,3,4,5,6,7,8,17,18]]:
    prob_cols.append(col)

bowler_stats_trimmed2 = bowler_stats[prob_cols]

In [54]:
bowler_stats_trimmed2[bowler_stats_trimmed2.bowler=='VG Arora']

,wkt_prob,one_prob,two_prob,three_prob,four_prob,six_prob,dot_prob,wide_prob,no_prob,bowler,innings,phase,nrr_phase,wkt_phase,num_innings,runs,balls,wkts,economy,strike_rate
2064,0.090909,0.247934,0.000000,0.000000,0.082645,0.165289,0.413223,0.000000,0.000000,VG Arora,2,death,crucial,crucial,2,19.0,11,1,10.363636,11.000000
2065,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,VG Arora,2,death,crucial,easy,1,5.0,0,0,inf,NaN
2066,0.000000,0.411765,0.058824,0.000000,0.235294,0.000000,0.176471,0.117647,0.000000,VG Arora,2,death,crucial,medium,2,27.0,15,0,10.800000,inf
2067,0.096774,0.283871,0.103226,0.000000,0.103226,0.000000,0.309677,0.103226,0.000000,VG Arora,2,death,easy,easy,6,39.0,31,3,7.548387,10.333333
2068,0.000000,0.466667,0.000000,0.000000,0.000000,0.133333,0.200000,0.200000,0.000000,VG Arora,2,death,easy,medium,3,22.0,12,0,11.000000,inf
2069,0.000000,0.333333,0.000000,0.000000,0.333333,0.000000,0.333333,0.000000,0.000000,VG Arora,2,death,moderate,easy,1,5.0,3,0,10.000000,inf
2070,0.333333,0.200000,0.000000,0.000000,0.000000,0.066667,0.333333,0.066667,0.000000,VG Arora,2,death,moderate,medium,2,10.0,9,3,6.666667,3.000000
2071,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,VG Arora,2,middle,crucial,crucial,1,1.0,1,0,6.000000,inf
2072,0.142857,0.367347,0.000000,0.000000,0.122449,0.000000,0.367347,0.000000,0.000000,VG Arora,2,middle,crucial,medium,2,7.0,7,1,6.000000,7.000000
2073,0.000000,0.600000,0.000000,0.000000,0.000000,0.000000,0.400000,0.000000,0.000000,VG Arora,2,middle,easy,crucial,2,6.0,10,0,3.600000,inf


In [55]:
bowler_stats_trimmed2.to_csv('/Users/roumyadas/Desktop/Data/t20_cricket_data/Data/created_data/bowler_stats_inn2.csv',
                           index=None)